# Observatory Distances

In this notebook, a dataframe is built based on the distances from each observatory to the Apophis in every time step.

In [1]:
import pandas as pd
import pylab as plt
import numpy as np

In [2]:
ephemeris = pd.read_csv("../ephemeris2.csv")
radio = pd.read_csv("../AllRadiotelescopes.csv")
print(radio)

    ID                      Name  \
0    5           Canberra DSS-43   
1   11                    Parkes   
2   10             Madrid DSS-63   
3    9                    Lovell   
4    8                Effelsberg   
5   12                 AVN-Ghana   
6    1                 Goldstone   
7    2                Green Bank   
8    3          Very Large Array   
9    0                   Arecibo   
10   4                      FAST   
11   6             Galenki RT-70   
12   7  Usuda Deep Space Center    

                                          Location  \
0        Tidbinbilla, Australian Capital Territory   
1              Parkes Observatory, New South Wales   
2                                           Madrid   
3      Jodrell Bank Observatory, Cheshire, England   
4   Bad M�nstereifel-Effelsberg near Bonn, Germany   
5                                   Kutunse, Ghana   
6                    Mojave Desert, California, US   
7                    Green Bank, West Virginia, US   
8        

## Distances

In [3]:
"ASTROPY"
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun
from astropy.constants import c
import astropy.units as u

In [4]:
import warnings
warnings.simplefilter('ignore', UserWarning)

In [5]:
# Obtains the Astropy Position Vector of given earth coordinates
pos = lambda x,h: EarthLocation(lon=x['Longitude']*u.deg,lat=x['Latitude']*u.deg,height=h*u.earthRad)

# Transforms an Astropy object into a simple 3-tuple vector
vec = lambda a:  np.array((a.x.value,
                  a.y.value,
                  a.z.value))

# Obtains the difference between two Astropy Vectors
vecdiff = lambda a,b: (a.x.value-b.x.value,
                       a.y.value-b.y.value,
                       a.z.value-b.z.value)

# Returns the magnitude of a vector
dist = lambda v: np.linalg.norm(v)

# Calculates the angle between two vectors in degrees
angle = lambda v1,v2: np.rad2deg( np.arccos( np.dot(v1,v2) / (dist(v1) * dist(v2)) ) )

In [6]:
distances = pd.DataFrame(columns=['datetime_jd','datetime_str'])
distances[['datetime_jd','datetime_str']] = ephemeris[['datetime_jd','datetime_str']]

def distance_to_apophis(obs,row):
    pos1 = pos(obs,0)
    posA = pos(row,row['delta (Rt)'])
    vec1 = vecdiff(posA,pos1) # Vector from obs1 to Apophis
    dist1 = dist(vec1)*u.earthRad # Magnitudes
    return dist1.value

for i,obs in radio.iterrows():
    name = obs['Name']
    obs_distances = []
    for j,row in ephemeris.iterrows():
            obs_distances += [distance_to_apophis(obs,row)]
    distances[name] = obs_distances
distances

,datetime_jd,datetime_str,Canberra DSS-43,Parkes,Madrid DSS-63,Lovell,Effelsberg,AVN-Ghana,Goldstone,Green Bank,Very Large Array,Arecibo,FAST,Galenki RT-70,Usuda Deep Space Center
0,2462240.063,2029-Apr-13 13:30,30.191080,30.184727,32.082872,32.000982,31.979115,32.034655,31.383546,31.832824,31.496637,31.941016,30.808688,30.857648,30.709430
1,2462240.066,2029-Apr-13 13:35,29.906999,29.900336,31.794780,31.713383,31.689575,31.744344,31.115917,31.561815,31.229158,31.670226,30.512297,30.568492,30.421322
2,2462240.069,2029-Apr-13 13:40,29.623267,29.616302,31.506395,31.425551,31.399803,31.453672,30.848328,31.290648,30.961666,31.399186,30.216104,30.279577,30.133507
3,2462240.073,2029-Apr-13 13:45,29.339662,29.332333,31.215977,31.136009,31.107734,31.159966,30.585436,31.022781,30.698796,31.131204,29.916401,29.989479,29.844901
4,2462240.076,2029-Apr-13 13:50,29.056749,29.049135,30.926927,30.847650,30.817438,30.868476,30.317908,30.751228,30.431242,30.859560,29.620703,29.701147,29.557786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2462240.740,2029-Apr-14 05:45,30.158195,30.123845,30.136885,30.036937,29.958390,30.318439,30.728870,30.770691,30.797336,30.994371,29.337427,29.521109,29.561232
196,2462240.743,2029-Apr-14 05:50,30.453067,30.419031,30.405548,30.308816,30.229456,30.581977,31.019876,31.052198,31.086152,31.271255,29.624777,29.813360,29.855791
197,2462240.747,2029-Apr-14 05:55,30.752157,30.718532,30.669805,30.577219,30.496889,30.839819,31.312654,31.332596,31.376047,31.545674,29.914056,30.108653,30.154117
198,2462240.750,2029-Apr-14 06:00,31.047444,31.014151,30.938422,30.849079,30.768034,31.103383,31.602969,31.613381,31.664120,31.821758,30.202240,30.401392,30.449159


In [7]:
distances.to_csv("../Data/observatory_distances_indexes.csv")

In [8]:
elevations = pd.DataFrame(columns=['datetime_jd','datetime_str'])
elevations[['datetime_jd','datetime_str']] = ephemeris[['datetime_jd','datetime_str']]

def elevation_angle(obs,row):
    pos1 = pos(obs,0)
    posA = pos(row,row['delta (Rt)'])
    vec1 = vecdiff(posA,pos1) # Vector from obs1 to Apophis
    zenith_angle = angle(vec(pos1),vec(posA)-vec(pos1))
    elevation = 90 - zenith_angle
    return elevation

for i,obs in radio.iterrows():
    name = obs['Code']
    obs_elevations = []
    for j,row in ephemeris.iterrows():
            obs_elevations += [elevation_angle(obs,row)]
    elevations[name] = obs_elevations
elevations

,datetime_jd,datetime_str,CDS,PRK,MDS,LOV,EFE,AVN,GOL,GBT,VLA,ARE,FAS,ECD,USU
0,2462240.063,2029-Apr-13 13:30,76.982866,78.735742,-67.140980,-57.385337,-55.169676,-60.853223,-13.522817,-42.628884,-20.241871,-51.478890,20.028456,17.056446,26.316442
1,2462240.066,2029-Apr-13 13:35,77.355138,79.265784,-66.379643,-56.882897,-54.507932,-59.995323,-14.413579,-43.531706,-21.171115,-52.563784,20.869098,17.441400,26.661918
2,2462240.069,2029-Apr-13 13:40,77.643703,79.706912,-65.600452,-56.362964,-53.834097,-59.119372,-15.309360,-44.434674,-22.104408,-53.650978,21.703018,17.812920,26.989775
3,2462240.073,2029-Apr-13 13:45,77.905974,80.148592,-64.577741,-55.679419,-52.956575,-57.944448,-16.486242,-45.614206,-23.327538,-55.062209,22.763656,18.257993,27.371096
4,2462240.076,2029-Apr-13 13:50,77.979076,80.338941,-63.763055,-55.122212,-52.257473,-57.032195,-17.392711,-46.516363,-24.269344,-56.154001,23.579858,18.596894,27.656370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2462240.740,2029-Apr-14 05:45,7.804390,9.806834,9.049244,14.967840,19.737776,-1.428749,-25.641391,-28.298351,-30.013649,-43.881196,76.475041,51.610234,47.921501
196,2462240.743,2029-Apr-14 05:50,7.104708,9.084382,9.877588,15.624829,20.464096,-0.323710,-26.157127,-28.215621,-30.406984,-43.426194,75.341196,50.716877,46.895704
197,2462240.747,2029-Apr-14 05:55,6.162587,8.112980,10.967742,16.494520,21.421014,1.108925,-26.786782,-28.061344,-30.872235,-42.784937,73.874075,49.565002,45.573769
198,2462240.750,2029-Apr-14 06:00,5.442741,7.370201,11.805398,17.158856,22.151747,2.215189,-27.262795,-27.931636,-31.219136,-42.275449,72.746342,48.668233,44.553975


In [9]:
elevations.to_csv("../Data/elevations.csv")